In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
def df_row_columns_cnt(df):
    return df.count(), len(df.columns)

In [0]:
def df_missing_value_cnt(df):
    missing_data = [(c, df.filter(isnull(col(c))).count()) for c in df.columns]
    df_missing = spark.createDataFrame(missing_data, ["column_name", "missing_count"])
    display(df_missing)

In [0]:
def missing_value_perc(df):
    total_rows = df.count()
    missing_perc = df.select([(count(when(col(c).isNull(), c)) / total_rows).alias(c) for c in df.columns])
    return display(missing_perc.selectExpr(
        "stack({},{}) as (column_name,missing_percentage)".format(
            len(df.columns),
            ', '.join([f"'{c}', {c}" for c in df.columns])
        )
    ))

In [0]:
def check_missing_perc_and_drop(df):
    total_rows=df.count()
    missing_values_less_75={}
    lst_cl=df.columns
    for col_name in lst_cl:
        missing_count=df.filter(col(col_name).isNull()).count()
        missing_perc=(missing_count/total_rows)*100
        if missing_perc<75:
            missing_values_less_75[col_name]=missing_perc
    else:
        df=df.drop(col_name)
    return display(df.limit(5))
       

In [0]:
def check_duplicates(df):
    a=df.select("*").distinct().count()
    b=df.count()
    if a==b:
        print("No duplicates found")
    else:
        print("Duplicates found")

In [0]:
def drop_duplicates(df):
    df=df.dropDuplicates()